In [226]:
#-*- coding: utf-8 -*-
import tensorflow as tf
import collections
import numpy as np
import sys
import time
import random

from tensorflow.python.ops import rnn_cell
from tensorflow.python.ops import seq2seq

In [197]:
batch_size    = 3 #minibatch size
seq_length    = 6 #RNN sequence length
learning_rate = 0.002
decay_rate    = 0.97
num_epochs    = 2

### 1. Data 준비

In [198]:
input_text = u"""윌리엄 세익스피어는 죽느냐 사느냐 그것이 문제라고 말했습니다.
                윌리엄 세익스피어는 죽느냐 사느냐 그것이 문제라고 말했습니다. 
                윌리엄 세익스피어는 죽느냐 사느냐 그것이 문제라고 말했습니다."""

In [159]:
x_text = input_text.split()

word_counts = collections.Counter(x_text)
# Mapping from index to word
words = [x[0] for x in word_counts.most_common()]
words = list(sorted(words))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(words)}
vocab_size = len(words)

words_index_list = np.array(list(map(vocab.get, x_text)))

In [160]:
print repr([word.encode(sys.stdout.encoding) for word in words]).decode('string-escape')

['그것이', '말했습니다.', '문제라고', '사느냐', '세익스피어는', '윌리엄', '죽느냐']


In [161]:
print repr([(item[0].encode(sys.stdout.encoding), item[1]) for item in vocab.items()]).decode('string-escape')

[('세익스피어는', 4), ('문제라고', 2), ('죽느냐', 6), ('사느냐', 3), ('말했습니다.', 1), ('그것이', 0), ('윌리엄', 5)]


In [162]:
print vocab_size

7


In [163]:
print words_index_list
print len(words_index_list)

[5 4 6 3 0 2 1 5 4 6 3 0 2 1 5 4 6 3 0 2 1]
21


In [164]:
num_batches = int(words_index_list.size / (batch_size * seq_length))
print num_batches

1


In [165]:
tensor_size = num_batches * batch_size * seq_length
print tensor_size

18


In [166]:
tensor = tensor[:tensor_size]

In [167]:
print tensor

[5 4 6 3 0 2 1 5 4 6 3 0 2 1 5 4 6 3]


In [168]:
xdata = tensor
ydata = np.copy(tensor)

ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]

print(xdata, len(xdata))
print(ydata, len(ydata))

(array([5, 4, 6, 3, 0, 2, 1, 5, 4, 6, 3, 0, 2, 1, 5, 4, 6, 3]), 18)
(array([4, 6, 3, 0, 2, 1, 5, 4, 6, 3, 0, 2, 1, 5, 4, 6, 3, 5]), 18)


In [169]:
print(xdata.reshape(batch_size, -1))
print(ydata.reshape(batch_size, -1))

[[5 4 6 3 0 2]
 [1 5 4 6 3 0]
 [2 1 5 4 6 3]]
[[4 6 3 0 2 1]
 [5 4 6 3 0 2]
 [1 5 4 6 3 5]]


In [170]:
x_batches = np.split(xdata.reshape(batch_size, -1), num_batches, 1) #Split an array into multiple sub-arrays.
y_batches = np.split(ydata.reshape(batch_size, -1), num_batches, 1)

print(x_batches)
print(y_batches)

[array([[5, 4, 6, 3, 0, 2],
       [1, 5, 4, 6, 3, 0],
       [2, 1, 5, 4, 6, 3]])]
[array([[4, 6, 3, 0, 2, 1],
       [5, 4, 6, 3, 0, 2],
       [1, 5, 4, 6, 3, 5]])]


In [171]:
def next_batch(pointer):
    x, y = x_batches[pointer], y_batches[pointer]
    return x, y

In [172]:
num_units = vocab_size #size of RNN hidden state

In [173]:
session = tf.InteractiveSession()

In [174]:
cell = rnn_cell.BasicRNNCell(num_units = num_units) # num_units = 7

In [175]:
initial_state = cell.zero_state(batch_size, tf.float32)
print initial_state.eval()

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]


In [211]:
input_data = tf.placeholder(tf.int32, [batch_size, seq_length])  #(3, 6)
targets = tf.placeholder(tf.int32, [batch_size, seq_length])     #(3, 6)

- word embedding

In [221]:
embedding = tf.Variable(tf.random_uniform([vocab_size, num_units], -1.0, 1.0))    #(7, 7)
print embedding;print

embedding_lookup = tf.nn.embedding_lookup(embedding, input_data)
print embedding_lookup;print

inputs_temp = tf.split(1, seq_length, embedding_lookup)
print inputs_temp;print

inputs = [tf.squeeze(input_, [1]) for input_ in inputs_temp]
print inputs;print

Tensor("Variable_6/read:0", shape=(7, 7), dtype=float32)

Tensor("embedding_lookup_7:0", shape=(3, 6, 7), dtype=float32)

[<tf.Tensor 'split_6:0' shape=(3, 1, 7) dtype=float32>, <tf.Tensor 'split_6:1' shape=(3, 1, 7) dtype=float32>, <tf.Tensor 'split_6:2' shape=(3, 1, 7) dtype=float32>, <tf.Tensor 'split_6:3' shape=(3, 1, 7) dtype=float32>, <tf.Tensor 'split_6:4' shape=(3, 1, 7) dtype=float32>, <tf.Tensor 'split_6:5' shape=(3, 1, 7) dtype=float32>]

[<tf.Tensor 'Squeeze_20:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'Squeeze_21:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'Squeeze_22:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'Squeeze_23:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'Squeeze_24:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'Squeeze_25:0' shape=(3, 7) dtype=float32>]



In [213]:
x, y = next_batch(0)

In [214]:
print x

[[5 4 6 3 0 2]
 [1 5 4 6 3 0]
 [2 1 5 4 6 3]]


In [215]:
tf.global_variables_initializer().run()
session.run(inputs, {input_data: x})

[array([[-0.41397262, -0.52421355, -0.26793003,  0.19232202,  0.48450661,
          0.98198891, -0.33509398],
        [-0.96061945,  0.07538652,  0.55114079, -0.8136189 , -0.66453218,
          0.29499269,  0.73907971],
        [-0.86935425, -0.04184699, -0.95249796, -0.292665  , -0.11538029,
          0.96657205, -0.79084444]], dtype=float32),
 array([[ 0.6182487 ,  0.58449626,  0.71015429,  0.71422744,  0.27648664,
         -0.83238077,  0.97287464],
        [-0.41397262, -0.52421355, -0.26793003,  0.19232202,  0.48450661,
          0.98198891, -0.33509398],
        [-0.96061945,  0.07538652,  0.55114079, -0.8136189 , -0.66453218,
          0.29499269,  0.73907971]], dtype=float32),
 array([[ 0.10757804,  0.44390416,  0.58406186,  0.54903483, -0.95282722,
          0.39436197, -0.90446091],
        [ 0.6182487 ,  0.58449626,  0.71015429,  0.71422744,  0.27648664,
         -0.83238077,  0.97287464],
        [-0.41397262, -0.52421355, -0.26793003,  0.19232202,  0.48450661,
          0.

In [235]:
with tf.variable_scope('rnnlm', reuse=True):
    softmax_w = tf.Variable(tf.random_uniform([num_units, vocab_size]))    #(7, 7)
    softmax_b = tf.Variable(tf.random_uniform([vocab_size]))  #7
    
    print softmax_w
    print softmax_b

    outputs, last_state = seq2seq.rnn_decoder(inputs, initial_state, cell)
    output = tf.reshape(tf.concat(1, outputs), [-1, num_units])    
    
    print outputs
    print output
   
    logits = tf.matmul(output, softmax_w) + softmax_b
    probs = tf.nn.softmax(logits)
    
    loss = seq2seq.sequence_loss_by_example([logits],
                                            [tf.reshape(targets, [-1])],
                                            [tf.ones([batch_size * seq_length])],
                                            vocab_size)
    lr = tf.Variable(0.0, trainable=False)
    total_loss = tf.reduce_sum(loss) / batch_size / seq_length
    optimizer = tf.train.AdamOptimizer(lr).minimize(total_loss)

Tensor("rnnlm_17/Variable/read:0", shape=(7, 7), dtype=float32)
Tensor("rnnlm_17/Variable_1/read:0", shape=(7,), dtype=float32)
[<tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell/Tanh:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell_1/Tanh:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell_2/Tanh:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell_3/Tanh:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell_4/Tanh:0' shape=(3, 7) dtype=float32>, <tf.Tensor 'rnnlm_17/rnn_decoder/BasicRNNCell_5/Tanh:0' shape=(3, 7) dtype=float32>]
Tensor("rnnlm_17/Reshape:0", shape=(18, 7), dtype=float32)


In [244]:
def sample(sess, words, vocab, probs, num=200, prime='', sampling_type=1, pick=0):
    
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

    def beam_search_pick(weights):
        probs[0] = weights
        samples, scores = BeamSearch(probs).beamsearch(None, vocab.get(prime), None, 2, len(weights), False)
        sampleweights = samples[np.argmax(scores)]
        t = np.cumsum(sampleweights)
        s = np.sum(sampleweights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))
    
    state = sess.run(cell.zero_state(1, tf.float32))
    print state
    
    if not len(prime) or prime == '':
        prime = random.choice(list(vocab.keys()))
    print prime
    
    for word in prime.split()[:-1]:
        print word
        x = np.zeros((1, 1))
        x[0, 0] = vocab.get(word,0)
        feed = {input_data: x}
        [state] = sess.run([last_state], feed)

    ret = prime
    word = prime.split()[-1]

    for n in range(num):
        x = np.zeros((1, 1))
        x[0, 0] = vocab.get(word,0)
        feed = {input_data: x}
        [probs, state] = sess.run([probs, last_state], feed)
        p = probs[0]

        if pick == 1:
            if sampling_type == 0:
                sample = np.argmax(p)
            elif sampling_type == 2:
                if word == '\n':
                    sample = weighted_pick(p)
                else:
                    sample = np.argmax(p)
            else: # sampling_type == 1 default:
                sample = weighted_pick(p)
        elif pick == 2:
            sample = beam_search_pick(p)

        pred = words[sample]
        ret += ' ' + pred
        word = pred
    return ret


In [245]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    for e in range(0, num_epochs):
        elasped_time = 0
        start = time.time()
        
        for i in range(0, num_batches):
            x, y = next_batch(i)
            sess.run(tf.assign(lr, learning_rate * (decay_rate ** e)))
            feed = {input_data: x, targets: y}
            sess.run(optimizer, feed)
            elasped_time = time.time() - start
            print elasped_time

    sample_text = sample(sess, words, vocab, probs)

0.111006975174
0.0787000656128
[[ 0.  0.  0.  0.  0.  0.  0.]]
윌리엄


ValueError: Cannot feed value of shape (1, 1) for Tensor u'Placeholder_7:0', which has shape '(3, 6)'